In [1]:
import os
import urllib.request
import pandas as pd
import ipywidgets as widgets
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

os.environ['WDM_LOG_LEVEL'] = '0'

In [2]:
def inicializarDriver(name):
    if name == ("google-chrome.desktop" or "brave-browser.desktop" or "chromium_chromium.desktop"):
        options = ChromeOptions()
        options.add_argument("--headless")

        if name == "google-chrome.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        elif name == "brave-browser.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()), options=options)
        elif name == "chromium_chromium.desktop":
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()), options=options)

    elif (name == "firefox.desktop"):
        options = FirefoxOptions()
        options.add_argument("--headless")
        driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install(), log_path=os.devnull), options=options)

    return driver

Setando a arbovirose

In [3]:
arbo_v = widgets.Combobox(
    placeholder = 'Selecione uma: ',
    options = ["Dengue", "Chikungunya", "Zika"],
    description = 'Arbovirose: ',
    ensure_option = True
)

display(arbo_v)

Combobox(value='', description='Arbovirose: ', ensure_option=True, options=('Dengue', 'Chikungunya', 'Zika'), …

In [4]:
arbo_v = str.lower(arbo_v.value)

if arbo_v == 'dengue':
    url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"
elif arbo_v == 'chikungunya':
    url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/chikun.def"
elif arbo_v == 'zika':
    url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/zika.def"

In [5]:
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html5lib')

linha = soup.find("div", {"class": "linha"}).find("select").select("option")
linha = list(map(lambda node: node.get_text().strip(), linha))

coluna = soup.find("div", {"class": "coluna"}).find("select").select("option")
coluna = list(map(lambda node: node.get_text().strip(), coluna))

periodo = soup.find("div", {"class": "periodo"}).find("select").select("option")
periodo = list(map(lambda node: node.get_text().strip(), periodo))

In [8]:
linha_disp = ['Município']
coluna_disp = ['Ano da Notificação', 'Mês da Notificação', 'Sexo', 'Fx Etária SINAN']

if arbo_v == 'dengue':
    coluna_disp.append('Ano do Óbito')

In [9]:
linha_valor = widgets.Combobox(
    placeholder = 'Selecione uma: ',
    options = linha_disp,
    description = 'Linha: ',
    ensure_option = True
)

display(linha_valor)

Combobox(value='', description='Linha: ', ensure_option=True, options=('Município',), placeholder='Selecione u…

In [10]:
coluna_valor = widgets.Combobox(
    placeholder = 'Selecione uma: ',
    options = coluna_disp,
    description = 'Coluna: ',
    ensure_option = True
)

display(coluna_valor)

Combobox(value='', description='Coluna: ', ensure_option=True, options=('Ano da Notificação', 'Mês da Notifica…

In [11]:
periodo_valor = widgets.SelectMultiple(
    options = periodo,
    value = periodo,
    description='Periodos',
    disabled=False
)

display(periodo_valor)

SelectMultiple(description='Periodos', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15), options=(…

In [12]:
linha_valor = linha_valor.value
coluna_valor = coluna_valor.value
periodo_valor = list(periodo_valor.value)

In [13]:
browser_name = ["google-chrome.desktop", "brave-browser.desktop", "chromium_chromium.desktop"]
i = 0
while (i < 3):
    try:
        driver = inicializarDriver(browser_name[i])
        i = 3
    except:
        i += 1

driver.set_page_load_timeout(100)
driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CLASS_NAME, 'mostra')))

if arbo_v == "dengue" or arbo_v == "chikungunya":
    # Linha
    driver.find_element(By.XPATH, f'/html/body/div/center/div/form/div[2]/div/div[1]/select/option[{linha.index(linha_valor)+1}]').click()
    # Coluna
    driver.find_element(By.XPATH, f'/html/body/div/center/div/form/div[2]/div/div[2]/select/option[{coluna.index(coluna_valor)+1}]').click()
    # Periodo
    driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()
    for i, item in enumerate(periodo):
        if item in periodo_valor:
            driver.find_element(By.XPATH, f'/html/body/div/center/div/form/div[3]/div/select/option[{i+1}]').click()
    # Mostra
    driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()
else:
    # Linha
    driver.find_element(By.XPATH, f'/html/body/center/div/form/div[2]/div/div[1]/select/option[{linha.index(linha_valor)+1}]').click()
    # Coluna
    driver.find_element(By.XPATH, f'/html/body/center/div/form/div[2]/div/div[2]/select/option[{coluna.index(coluna_valor)+1}]').click()
    # Periodo
    driver.find_element(By.XPATH, '/html/body/center/div/form/div[3]/div/select/option[1]').click()
    for i, item in enumerate(periodo):
        if item in periodo_valor:
            driver.find_element(By.XPATH, f'/html/body/center/div/form/div[3]/div/select/option[{i+1}]').click()

    # Mostra
    driver.find_element(By.XPATH, '/html/body/center/div/form/div[4]/div[2]/div[2]/input[1]').click()

html = driver.page_source

driver.quit()

In [14]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [15]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [16]:
aux = []
for item in df.iloc[:, 0]:
    aux.append(item.split(" ", 1))

aux = pd.DataFrame(aux)

df["Código"] = aux[0]
df["Município"] = aux[1]

aux = list(df.columns)
aux.insert(0, aux.pop())
df = df[aux]

In [17]:
df.iloc[:, 2:] = df.iloc[:, 2:].astype(str).replace('\.', '', regex=True)
df.iloc[:, 2:] = df.iloc[:, 2:].astype(str).replace('-', '0')

In [18]:
df.head()

,Código,Município,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez
0,290010,Abaíra,0,0,0,0,0,0,0,1,0,0,0,0
1,290020,Abaré,0,0,0,1,0,0,0,0,0,0,1,0
2,290030,Acajutiba,0,0,0,0,0,1,0,1,0,0,0,1
3,290035,Adustina,5,0,0,0,3,0,0,0,1,0,0,0
4,290040,Água Fria,0,0,0,0,1,0,0,0,0,0,0,0


In [19]:
aux = ''
for i, item in enumerate(periodo_valor):
    aux += '_' + item
aux

if arbo_v == "dengue":
    df.to_csv(f"../scr/dengue/{linha_valor}_{coluna_valor}{aux}.csv", encoding='utf-8')
elif arbo_v == "chikungunya":
    df.to_csv(f"../scr/chikungunya/{linha_valor}_{coluna_valor}{aux}.csv", encoding='utf-8')
else:
    df.to_csv(f"../scr/zika/{linha_valor}_{coluna_valor}{aux}.csv", encoding='utf-8')